# Introduction to MPI on Amazon SageMaker

Message Passing Interface (MPI) is the fundamental communication protocol for programming parallel computer programs. See its [wiki page](https://en.wikipedia.org/wiki/Message_Passing_Interface). [Open MPI](https://www.open-mpi.org/projects/user-docs/) is the implementation that's used as a basic building block for distributed training system. 

In Python programs, you can interact with Open MPI APIs via [mpi4py](https://mpi4py.readthedocs.io/en/stable/overview.html) and easily convert your single-process python program into a parallel python program. 

Parallel processes can exist on one host (e.g. one EC2 instance) or multiple hosts (e.g. many EC2 instances). It's trivial to set up a parallel cluster (comm world, in MPI parlance) on one host via Open MPI, but it is less straight-forward to set up an MPI comm world across multiple instances. 

SageMaker does it for you. In this tutorial, you will go through a few basic (but exceeding important) [MPI communications](https://mpi4py.readthedocs.io/en/stable/tutorial.html) on SageMaker with **multiple instances** and you will verify that parallel processes across instances are indeed talking to each other. Those basic communications are the fundamental building blocks for distributed training.

## Environment 
We assume Open MPI and mpi4py have been installed in your environment. This is the case for SageMaker Notebook Instance or Studio. 

## Inspect the Python Program

In [1]:
!pygmentize mpi_demo.py

from mpi4py import MPI
import numpy as np
import time

comm = MPI.COMM_WORLD
size = comm.Get_size()
rank = comm.Get_rank()

if rank == 0:
    print("Number of MPI processes that will talk to each other: ", size)


def point_to_point():
    """Point to point communication
    Send a numpy array (buffer like object) from rank 0 to rank 1
    """
    if rank == 0:
        print('point to point')
        data = np.array([0, 1, 2], dtype=np.intc) # int in C

        # remember the difference between
        # Upper case API and lower case API
        # Basically uppper case API directly calls C API
        # so it is fast
        # checkout https://mpi4py.readthedocs.io/en/stable/

        comm.Send([data, MPI.INT], dest=1)
    elif rank == 1:
        print(f'Hello I am rank {rank}')
        data = np.empty(3, dtype=np.intc)
        comm.Recv([data, MPI.INT], source=0)
        print('I received some data:', data)
        
    if rank == 0:
        time.sleep(1) # give some buffer time for e

See the program in action with 2 parallel processes on your current environment. Make sure you have at least 2 cores.

In [2]:
!mpirun -np 2 python mpi_demo.py

Number of MPI processes that will talk to each other:  2
point to point
Hello I am rank 1
I received some data: [0 1 2]
Broadcasting from rank 0
Data at rank 0 [0 1 2 3 4 5 6 7 8 9]
Data at rank 1 [0 1 2 3 4 5 6 7 8 9]
Gather and reduce
I am rank 0, data I gathered is: [[0 0 0 0 0 0 0 0 0 0]
 [1 1 1 1 1 1 1 1 1 1]]
I am rank 0, my avg is: [0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5]
I am rank 1, my avg is: [0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5]


## Scale it on SageMaker
You can run the above program with $n$ processes per host across $N$ hosts on SageMaker (and get a comm world of size $n\times N$). In the remaining of this notebook, you will use SageMaker TensorFlow deep learning container to run the above program. There is no particular reason for the choice, all SageMaker deep learning containers have Open MPI installed. So feel free to replace it with your favorite DLC. 

Check out the [SageMaker Python SDK Docs](https://sagemaker.readthedocs.io/en/stable/api/training/smd_model_parallel_general.html?highlight=mpi%20paramters#mpi-parameters) for the parameters needed to set up a distributed training job with MPI. 

In [3]:
import sagemaker
from sagemaker import get_execution_role
from sagemaker.tensorflow import TensorFlow

role = get_execution_role()

# Running 2 processes per host
# if we use 3 instances,
# then we should see 12 MPI processes

distribution = {"mpi": {"enabled": True, "processes_per_host": 2}}

tfest = TensorFlow(
    entry_point="mpi_demo.py",
    role=role,
    framework_version="2.3.0",
    distribution=distribution,
    py_version="py37",
    instance_count=3,
    instance_type="ml.c5.2xlarge",  # 8 cores
    output_path="s3://" + sagemaker.Session().default_bucket() + "/" + "mpi",
)

In [4]:
tfest.fit()

2021-05-11 19:56:11 Starting - Starting the training job...
2021-05-11 19:56:35 Starting - Launching requested ML instancesProfilerReport-1620762971: InProgress
......
2021-05-11 19:57:36 Starting - Preparing the instances for training......
2021-05-11 19:58:36 Downloading - Downloading input data...
2021-05-11 19:59:09 Training - Training image download completed. Training in progress..2021-05-11 19:59:14,435 sagemaker-training-toolkit INFO     Imported framework sagemaker_tensorflow_container.training
2021-05-11 19:59:14,442 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2021-05-11 19:59:14,937 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2021-05-11 19:59:14,951 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2021-05-11 19:59:14,959 sagemaker-training-toolkit INFO     Starting MPI run as worker node.
2021-05-11 19:59:14,959 sagemaker-training-toolkit INFO     Waiting for MPI M

The stdout "Number of MPI processes that will talk to each other:  6" indicates that the processes on all hosts are included in the comm world. 

## Conclusion
In this notebook, you went through some fundamental MPI operations, which are the bare bone of inner workings of many distributed training frameworks. You did that on SageMaker with multiple instances. You can scale up this set up to include more instances in a real ML project. 